In [1]:
!pip install --quiet transformers datasets

# Load datavie

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
from datasets import load_dataset, Audio

data_name = 'thinhlpg/vivoice-distil'
data = load_dataset(data_name, split='train[20%:25%]')

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/56761 [00:00<?, ? examples/s]

In [10]:
# resampling rate
data = data.cast_column("audio", Audio(sampling_rate=16000))

# Load m2m-100 model

In [2]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model_m2m = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_1.2B")
tokenizer_m2m = M2M100Tokenizer.from_pretrained("facebook/m2m100_1.2B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
vi_text = "Con muốn mua ít đồ."

tokenizer_m2m.src_lang = "vi"
encoded_hi = tokenizer_m2m(vi_text, return_tensors="pt")
generated_tokens = model_m2m.generate(**encoded_hi, forced_bos_token_id=tokenizer_m2m.get_lang_id("de"))
tokenizer_m2m.batch_decode(generated_tokens, skip_special_tokens=True)

['Ich möchte ein paar Sachen kaufen.']

# Load whisper model is fine-tuned with datavie

In [4]:
# Load model directly
from transformers import WhisperProcessor, WhisperForConditionalGeneration

processor_whisper = WhisperProcessor.from_pretrained("QuanHcmus/whisper-base-datavie")
model_whisper = WhisperForConditionalGeneration.from_pretrained("QuanHcmus/whisper-base-datavie")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
example = data[0]

forced_decoder_ids = processor_whisper.get_decoder_prompt_ids(language="vietnamese", task="transcribe")
input_features = processor_whisper(example["audio"]["array"], sampling_rate=example['audio']["sampling_rate"], return_tensors="pt").input_features
#generate token ids
predicted_ids = model_whisper.generate(input_features, forced_decoder_ids=forced_decoder_ids)
# decode token ids to text
transcription = processor_whisper.batch_decode(predicted_ids, skip_special_tokens=True)
transcription

['Ngày hai mươi chín tháng hai theo dở địa phương, trong phiên điều trần tại Ủy ban Quân sự Hạ Viện Mỹ.']

# Combine models and test

In [11]:
data_test_1 = data[0]
data_test_2 = data[1]
data_test_3 = data[2]
data_test_4 = data[3]
data_test_5 = data[4]

### Test first sample

In [19]:
def pipeline(data):
  # transcription audio
  forced_decoder_ids = processor_whisper.get_decoder_prompt_ids(language="vietnamese", task="transcribe")
  input_features = processor_whisper(data["audio"]["array"], sampling_rate=data['audio']["sampling_rate"], return_tensors="pt").input_features
  predicted_ids = model_whisper.generate(input_features, forced_decoder_ids=forced_decoder_ids)
  transcription = processor_whisper.batch_decode(predicted_ids, skip_special_tokens=True)
  # translate result
  tokenizer_m2m.src_lang = "vi"
  encoded_hi = tokenizer_m2m(transcription[0], return_tensors="pt")
  generated_tokens = model_m2m.generate(**encoded_hi, forced_bos_token_id=tokenizer_m2m.get_lang_id("de"))
  translation = tokenizer_m2m.batch_decode(generated_tokens, skip_special_tokens=True)
  return transcription[0], translation[0]

In [20]:
from IPython.display import Audio

Audio(data_test_1["audio"]["array"], rate=data_test_1["audio"]["sampling_rate"])

In [21]:
data_test_1['text']

'Ngày hai mươi chín tháng hai, theo giới địa phương, trong phiên điều trần tại Ủy ban Quân sự Hạ viện Mỹ.'

In [22]:
transcription, transtale = pipeline(data_test_1)

In [24]:
print("Vietnamese:", transcription)
print("German:", transtale)

Vietnamese: Ngày hai mươi chín tháng hai theo dở địa phương, trong phiên điều trần tại Ủy ban Quân sự Hạ Viện Mỹ.
German: Am 29. Februar folgten lokale Unruhen bei einer Anhörung im Militärkomitee des US-Kongresses.
